In [ ]:
!pip install protobuf==4.25.5 chromadb sentence-transformers transformers torch fastapi uvicorn pandas pyngrok
!pip install accelerate bitsandbytes sentencepiece

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
import torch
from transformers import BitsAndBytesConfig, pipeline, AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import os
from google.colab import userdata
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
import base64
from io import BytesIO

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

# Load dataset
dataset = pd.read_excel("/content/BreastCancer_GP.xlsx")
questions = dataset['السؤال'].tolist()
answers = dataset['الاجابه'].tolist()

print("Dataset loaded successfully!")
print(f"Number of questions: {len(questions)}")

Dataset loaded successfully!
Number of questions: 2215


In [ ]:
client = chromadb.Client()
# Corrected the method name from get_or_or_create_collection to get_or_create_collection
collection = client.get_or_create_collection(name="breast_cancer_qa")
# Explicitly pass the Hugging Face token to the SentenceTransformer
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large", token=os.environ.get('HF_TOKEN')).to("cuda")

# Add Q&A to ChromaDB
for i, (question, answer) in enumerate(zip(questions, answers)):
    embedding = embedding_model.encode(question, convert_to_tensor=True).cpu().numpy()
    collection.add(embeddings=[embedding], metadatas=[{"السؤال": question, "الاجابة": answer}], ids=[str(i)])

print("ChromaDB initialized with Q&A embeddings!")

ChromaDB initialized with Q&A embeddings!


In [ ]:
print("Number of items in the collection:", collection.count())

Number of items in the collection: 2215


In [ ]:
!pip install autoawq

In [ ]:
# Load AYA model
aya_model_id = "Orion-zhen/aya-expanse-8b-AWQ"
tokenizer = AutoTokenizer.from_pretrained(aya_model_id, token=os.environ['HF_TOKEN'])
model = AutoModelForCausalLM.from_pretrained(
    aya_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=os.environ['HF_TOKEN']
)

def generate_aya_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=500, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

`torch.bfloat16` is not supported for AWQ CUDA kernels yet. Casting to `torch.float16`.
/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Test AYA
test_response = generate_aya_response("ما هي أسباب سرطان الثدي؟")
print("AYA test response:", test_response)

AYA test response: ما هي أسباب سرطان الثدي؟ - سما مصر
الرئيسية » صحة و طب » ما هي أسباب سرطان الثدي؟
ما هي أسباب سرطان الثدي؟
نشر قبل : 3 سنوات 14 يومًا بتاريخ 17 نوفمبر، 2016 1:47 م
سرطان الثدي هو أحد أكثر أنواع السرطان شيوعاً بين النساء، وهو مرض يصيب خلايا الثدي، ويمكن أن ينتشر إلى أجزاء أخرى من الجسم.
هناك العديد من العوامل التي قد تزيد من خطر الإصابة بسرطان الثدي، ولكن لا يوجد سبب مؤكد للإصابة بهذا المرض.
فيما يلي بعض العوامل التي قد تزيد من خطر الإصابة بسرطان الثدي:
العمر: يزيد خطر الإصابة بسرطان الثدي مع تقدم العمر، حيث أن 75% من الحالات تحدث لدى النساء اللاتي تزيد أعمارهن عن 50 عاماً.
الجنس: سرطان الثدي أكثر شيوعاً بين النساء، ولكن يمكن أن يصيب الرجال أيضاً على الرغم من ندرة الإصابة بهم.
التاريخ العائلي: إذا كان لديك أقارب من الدرجة الأولى مصابون بسرطان الثدي، فهناك احتمال أكبر للإصابة به.
الوراثة: هناك بعض الطفرات الجينية التي تزيد من خطر الإصابة بسرطان الثدي، مثل طفرات الجينات BRCA1 و BRCA2.
التعرض للإشعاع: التعرض للإشعاع، خاصة في سن مبكرة، يزيد من خطر الإصابة بسرطان الثدي.
ال

In [ ]:
 !pip install sacremoses

In [ ]:
model_id = 'google/medgemma-4b-it'
model_kwargs = {
    'torch_dtype': torch.bfloat16,
    'device_map': 'auto',
    'quantization_config': BitsAndBytesConfig(load_in_4bit=True)
}

pipe = pipeline('image-text-to-text', model=model_id, model_kwargs=model_kwargs, token=os.environ['HF_TOKEN'])
pipe.model.generation_config.do_sample = False
ar_to_en_translator = pipeline('translation', model='Helsinki-NLP/opus-mt-ar-en', device_map='auto')
en_to_ar_translator = pipeline('translation', model='Helsinki-NLP/opus-mt-en-ar', device_map='auto')

print("MedGemma and translators initialized!")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0


MedGemma and translators initialized!


In [ ]:
test_translation = ar_to_en_translator("الصورة الشعاعية")[0]['translation_text']
print("Translation test:", test_translation)

Translation test: Radiography


In [ ]:
def process_query(arabic_prompt, image=None):
    if not arabic_prompt:
        return "يرجى إدخال سؤال"

    if image:  # Multimodal query
        try:
            en_prompt = ar_to_en_translator(arabic_prompt)[0]['translation_text']
            messages = [
                {'role': 'system', 'content': [{'type': 'text', 'text': 'You are an expert radiologist.'}]},
                {'role': 'user', 'content': [{'type': 'text', 'text': en_prompt}, {'type': 'image', 'image': image}]}
            ]
            with torch.inference_mode():
                output = pipe(messages, max_new_tokens=500)
            en_response = output[0]['generated_text'][-1]['content']
            arabic_response = en_to_ar_translator(en_response)[0]['translation_text']
            return arabic_response
        except Exception as e:
            return f"خطأ في معالجة الصورة: {str(e)}"
    else:  # Text-only query
        try:
            # Try ChromaDB first
            embedding = embedding_model.encode(arabic_prompt, convert_to_tensor=True).cpu().numpy()
            results = collection.query(query_embeddings=[embedding], n_results=1)
            if results['metadatas'][0][0].get('الاجابة'):
                return results['metadatas'][0][0]['الاجابة']
        except:
            pass
        # Fallback to AYA
        return generate_aya_response(arabic_prompt)

In [ ]:
# Test the function
print("Text-only test:", process_query("ما أعراض سرطان الثدي؟"))

Text-only test: تشمل الأعراض ظهور كتلة في الثدي، تغييرات في شكل الثدي، أو إفرازات غير طبيعية من الحلمة.


In [ ]:
from fastapi import FastAPI, File, UploadFile, Form
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/ask")
async def ask_question(question: str = Form(...), image: UploadFile = File(None)):
    try:
        if image:
            image_data = await image.read()
            image = Image.open(BytesIO(image_data)).convert("RGB")
            response = process_query(question, image)
        else:
            response = process_query(question)
        return JSONResponse({"الجواب": response})
    except Exception as e:
        return JSONResponse({"الجواب": f"خطأ: {str(e)}"})

print("FastAPI endpoint with CORS set up!")

FastAPI endpoint with CORS set up!


In [ ]:
!pip install pyngrok
print("pyngrok installed!")

pyngrok installed!


In [ ]:
!pip install --upgrade websockets
print("websockets updated!")

websockets updated!


In [ ]:
from pyngrok import ngrok
from google.colab import userdata

ngrok.set_auth_token(userdata.get('NGROK_TOKEN'))
public_url = ngrok.connect(8000)
print(f"Backend URL: {public_url}")

Backend URL: NgrokTunnel: "https://f41d36d48364.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()


In [ ]:
import uvicorn
uvicorn.run(app, host="0.0.0.0", port=8000)

/usr/local/lib/python3.11/dist-packages/websockets/legacy/__init__.py:6: DeprecationWarning: websockets.legacy is deprecated; see https://websockets.readthedocs.io/en/stable/howto/upgrade.html for upgrade instructions
  warnings.warn(  # deprecated in 14.0 - 2024-11-09
/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
INFO:     Started server process [19427]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
